In [31]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import pandas as pd
import requests
import os

In [32]:
# URL of page to be scraped
url = 'http://www.espn.com/nba/salaries'

In [33]:
# Retrieve page with the requests module
response = requests.get(url)

In [34]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [35]:
# Examine the results, then determine element that contains sought info
# print(soup.prettify())

In [36]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)

In [37]:
dropdown_list = []
for season in browser.find_by_xpath('//select[@class="tablesm"]//option'):
    if season.value == '#':
        continue
        
    dropdown_list += [season.value]
#     browser.find_by_xpath(f'//select[@class="tablesm"]//option[@value="{season.value}"]').click()
#     print(season.value)
    
# browser.find_by_xpath(f'//select[@class="tablesm"]//option[@value="{season.value}"]').click()
dropdown_list

['//www.espn.com/nba/salaries',
 '//www.espn.com/nba/salaries/_/year/2018',
 '//www.espn.com/nba/salaries/_/year/2017',
 '//www.espn.com/nba/salaries/_/year/2016',
 '//www.espn.com/nba/salaries/_/year/2015',
 '//www.espn.com/nba/salaries/_/year/2014',
 '//www.espn.com/nba/salaries/_/year/2013',
 '//www.espn.com/nba/salaries/_/year/2012',
 '//www.espn.com/nba/salaries/_/year/2011',
 '//www.espn.com/nba/salaries/_/year/2010',
 '//www.espn.com/nba/salaries/_/year/2009',
 '//www.espn.com/nba/salaries/_/year/2008',
 '//www.espn.com/nba/salaries/_/year/2007',
 '//www.espn.com/nba/salaries/_/year/2006',
 '//www.espn.com/nba/salaries/_/year/2005',
 '//www.espn.com/nba/salaries/_/year/2004',
 '//www.espn.com/nba/salaries/_/year/2003',
 '//www.espn.com/nba/salaries/_/year/2002',
 '//www.espn.com/nba/salaries/_/year/2001',
 '//www.espn.com/nba/salaries/_/year/2000']

In [38]:
df = pd.DataFrame(columns=['RK', 'Name', 'Team', 'Salary', 'Season'])

for season in dropdown_list:
    
    browser.find_by_xpath(f'//select[@class="tablesm"]//option[@value="{season}"]').first.click()

    a = True
    while a:
        tables = pd.read_html(browser.url)
        page_df = tables[0]
        page_df.columns = ['RK', 'Name', 'Team', 'Salary']
        page_df['Season'] = soup.find('div', class_="mod-header stathead").text.split(' ')[0]
        df = df.append(page_df, ignore_index = True)

        # Click the 'Next' button on each page
        try:
            browser.find_by_xpath('//div[@class="jcarousel-next"]').click()

        except :
            a = False

df

,RK,Name,Team,Salary,Season
0,RK,NAME,TEAM,SALARY,2018-2019
1,1,"Stephen Curry, PG",Golden State Warriors,"$37,457,154",2018-2019
2,2,"Russell Westbrook, PG",Oklahoma City Thunder,"$35,654,150",2018-2019
3,3,"Chris Paul, PG",Houston Rockets,"$35,654,150",2018-2019
4,4,"Blake Griffin, PF",Detroit Pistons,"$32,088,932",2018-2019
5,5,"Gordon Hayward, SF",Boston Celtics,"$31,214,295",2018-2019
6,6,"Kyle Lowry, PG",Toronto Raptors,"$31,200,000",2018-2019
7,7,"Paul George, SF",Oklahoma City Thunder,"$30,560,700",2018-2019
8,8,"Mike Conley, PG",Memphis Grizzlies,"$30,521,115",2018-2019
9,9,"James Harden, PG",Houston Rockets,"$30,421,854",2018-2019


In [39]:
# Clean rows

# Split name from position and create new column

# Reorder columns


In [40]:
df.to_csv('Resources\salary.csv')